# Classification Surrogate Tests

We are interested in testing whether or not a surrogate model can correctly identify unknown constraints based on categorical criteria with classification surrogates. Essentially, we want to account for scenarios where specialists can look at a set of experiments and label outcomes as 'acceptable', 'unacceptable', 'ideal', etc. 

This involves new models that produce `CategoricalOutput`'s rather than continuous outputs. Mathematically, if $g_{\theta}:\mathbb{R}^d\to[0,1]^c$ represents the function governed by learnable parameters $\theta$ which outputs a probability vector over $c$ potential classes (i.e. for input $x\in\mathbb{R}^d$, $g_{\theta}(x)^\top\mathbf{1}=1$ where $\mathbf{1}$ is the vector of all 1's) and we have acceptibility criteria for the corresponding classes given by $a\in\{0,1\}^c$, we can compute the scalar output $g_{\theta}(x)^\top a\in[0,1]$ which represents the expected value of acceptance as an objective value to be passed in as a constrained function.

In this script, we look at the [Rosenbrock function constrained to a disk](https://en.wikipedia.org/wiki/Test_functions_for_optimization#cite_note-12) which attains a global minima at $(x_0^*,x_1^*)=(1.0, 1.0)$. To facilitate testing the functionality offered by BoFire, we label all points inside of the circle $x_0^2+x_1^2\le2$ as 'acceptable' and further label anything inside of the intersection of this circle and the circle $(x_0-1)^2+(x_1-1)^2\le1.0$ as 'ideal'; points lying outside of these two locations are labeled as "unacceptable."

In [ ]:
# Import packages
import pandas as pd

import bofire.strategies.api as strategies
from bofire.data_models.api import Domain, Inputs, Outputs
from bofire.data_models.features.api import (
    CategoricalInput,
    CategoricalOutput,
    ContinuousInput,
    ContinuousOutput,
)
from bofire.data_models.objectives.api import (
    ConstrainedCategoricalObjective,
    MinimizeObjective,
)

## Manual setup of the optimization domain

The following cells show how to manually setup the optimization problem in BoFire for didactic purposes.

In [ ]:
# Write helper functions which give the objective and the constraints
def rosenbrock(x: pd.Series) -> pd.Series:
    assert "x_0" in x.columns
    assert "x_1" in x.columns
    return (1 - x["x_0"]) ** 2 + 100 * (x["x_1"] - x["x_0"] ** 2) ** 2


def constraints(x: pd.Series) -> pd.Series:
    assert "x_0" in x.columns
    assert "x_1" in x.columns
    feasiblity_vector = []
    for _, row in x.iterrows():
        if (row["x_0"] ** 2 + row["x_1"] ** 2 <= 2.0) and (
            (row["x_0"] - 1.0) ** 2 + (row["x_1"] - 1.0) ** 2 <= 1.0
        ):
            feasiblity_vector.append("ideal")
        elif row["x_0"] ** 2 + row["x_1"] ** 2 <= 2.0:
            feasiblity_vector.append("acceptable")
        else:
            feasiblity_vector.append("unacceptable")
    return feasiblity_vector

In [ ]:
# Set-up the inputs and outputs, use categorical domain just as an example
input_features = Inputs(
    features=[ContinuousInput(key=f"x_{i}", bounds=(-1.75, 1.75)) for i in range(2)]
    + [CategoricalInput(key="x_3", categories=["0", "1"], allowed=[True, True])],
)

# here the minimize objective is used, if you want to maximize you have to use the maximize objective.
output_features = Outputs(
    features=[
        ContinuousOutput(key="f_0", objective=MinimizeObjective(w=1.0)),
        CategoricalOutput(
            key="f_1",
            categories=["unacceptable", "acceptable", "ideal"],
            objective=ConstrainedCategoricalObjective(
                categories=["unacceptable", "acceptable", "ideal"],
                desirability=[False, True, True],
            ),
        ),  # This function will be associated with learning the categories
    ],
)

# Create domain
domain1 = Domain(inputs=input_features, outputs=output_features)

# Sample random points
sample_df = domain1.inputs.sample(100)

# Write a function which outputs one continuous variable and another discrete based on some logic
sample_df["f_0"] = rosenbrock(x=sample_df)
sample_df["f_1"] = constraints(x=sample_df)

sample_df.head(5)

,x_0,x_1,x_3,f_0,f_1
0,1.099949,1.336708,1,1.618309,unacceptable
1,0.950826,-1.417155,1,538.811012,unacceptable
2,-0.105881,-0.842408,1,74.089432,acceptable
3,-1.316266,0.064464,0,283.618091,acceptable
4,-1.274571,0.474821,0,137.357028,acceptable


In [ ]:
# Plot the sample df
import math

import plotly.express as px


fig = px.scatter(
    sample_df,
    x="x_0",
    y="x_1",
    color="f_1",
    width=550,
    height=525,
    title="Samples with labels",
)
fig.add_shape(
    type="circle",
    xref="x",
    yref="y",
    opacity=0.1,
    fillcolor="red",
    x0=-math.sqrt(2),
    y0=-math.sqrt(2),
    x1=math.sqrt(2),
    y1=math.sqrt(2),
    line_color="red",
)
fig.add_shape(
    type="circle",
    xref="x",
    yref="y",
    opacity=0.2,
    fillcolor="LightSeaGreen",
    x0=0,
    y0=0,
    x1=2,
    y1=2,
    line_color="LightSeaGreen",
)
fig.show()

## Evaluate the classification model performance (outside of the optimization procedure)

In [ ]:
# Import packages
import bofire.surrogates.api as surrogates
from bofire.data_models.surrogates.api import ClassificationMLPEnsemble
from bofire.surrogates.diagnostics import ClassificationMetricsEnum


# Instantiate the surrogate data model
surrogate_data = ClassificationMLPEnsemble(
    inputs=domain1.inputs,
    outputs=Outputs(features=[domain1.outputs.get_by_key("f_1")]),
    lr=0.03,
    n_epochs=100,
    hidden_layer_sizes=(
        4,
        2,
    ),
    weight_decay=0.0,
    batch_size=10,
    activation="tanh",
)
surrogate = surrogates.map(surrogate_data)

# Fit the surrogate to the classification data
cv_df = sample_df.drop(["f_0"], axis=1)
cv_df["valid_f_1"] = 1
cv_train, cv_test, _ = surrogate.cross_validate(cv_df, folds=3)

In [ ]:
# Print training performance
cv_train.get_metrics(
    metrics=ClassificationMetricsEnum,
    combine_folds=True,
)

,ACCURACY,F1
0,0.875,0.875


In [ ]:
# Print test performance
cv_test.get_metrics(
    metrics=ClassificationMetricsEnum,
    combine_folds=True,
)

,ACCURACY,F1
0,0.7,0.7


## Setup strategy and ask for candidates

Now we setup a `SoboStrategy` for generating candidates, the categorical output is modelled using the surrogate from above. The categorical output is modelled as an output constraint in the acquistion function optimization (constrained expected improvement). For more details have a look at this notebook: https://github.com/pytorch/botorch/blob/main/notebooks_community/clf_constrained_bo.ipynb and/or this paper: https://arxiv.org/abs/2402.07692.


In [ ]:
from bofire.data_models.acquisition_functions.api import qLogEI
from bofire.data_models.strategies.api import SoboStrategy
from bofire.data_models.surrogates.api import BotorchSurrogates


strategy_data = SoboStrategy(
    domain=domain1,
    acquisition_function=qLogEI(),
    surrogate_specs=BotorchSurrogates(
        surrogates=[surrogate_data],
    ),
)

strategy = strategies.map(strategy_data)

strategy.tell(sample_df)

In [ ]:
candidates = strategy.ask(10)
candidates

/Users/j30607/sandbox/botorch/botorch/optim/optimize.py:652: RuntimeWarning:

Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.



,x_0,x_1,x_3,f_1_pred,f_1_sd,f_1_unacceptable_prob,f_1_acceptable_prob,f_1_ideal_prob,f_0_pred,f_1_unacceptable_sd,f_1_acceptable_sd,f_1_ideal_sd,f_0_sd,f_0_des,f_1_des
0,-0.440780,0.191191,0,ideal,0.0,0.839229,0.098724,0.901083,0.000194,9.291584,0.203003,0.202908,0.000249,-0.000194,0.999806
1,-0.124262,0.037185,0,ideal,0.0,0.310786,0.100112,0.899684,0.000204,6.989385,0.202408,0.202312,0.000261,-0.000204,0.999796
2,0.528477,0.287229,1,ideal,0.0,0.328823,0.044223,0.422612,0.533165,5.849975,0.080326,0.514594,0.492027,-0.533165,0.466835
3,1.336751,1.750000,1,acceptable,0.0,0.604687,0.606031,0.002018,0.391951,8.187463,0.537966,0.004308,0.535448,-0.391951,0.608049
4,-0.584689,0.359022,0,unacceptable,0.0,1.484078,0.099049,0.900751,0.000200,8.914246,0.203335,0.203237,0.000258,-0.000200,0.999800
5,0.098145,-0.021054,0,ideal,0.0,0.481963,0.140105,0.859511,0.000384,5.748780,0.207016,0.207099,0.000598,-0.000384,0.999616
6,-0.258972,0.066109,0,ideal,0.0,0.406384,0.098858,0.900948,0.000194,8.132467,0.202901,0.202808,0.000248,-0.000194,0.999806
7,0.803090,0.651708,1,acceptable,0.0,0.205122,0.217384,0.197362,0.585254,4.479931,0.433316,0.413672,0.530183,-0.585254,0.414746
8,0.151093,0.004787,1,ideal,0.0,0.806973,0.098383,0.901426,0.000191,5.392008,0.202613,0.202520,0.000245,-0.000191,0.999809
9,0.333958,0.113097,1,ideal,0.0,0.659254,0.102805,0.896970,0.000226,4.785143,0.212465,0.212368,0.000276,-0.000226,0.999774


## Check classification of proposed candidates

Use the logic from above to verify the classification values

In [ ]:
# Append to the candidates
candidates["f_1_true"] = constraints(x=candidates)

In [ ]:
# Print results
candidates[["x_0", "x_1", "f_1_pred", "f_1_true"]]

,x_0,x_1,f_1_pred,f_1_true
0,-0.440780,0.191191,ideal,acceptable
1,-0.124262,0.037185,ideal,acceptable
2,0.528477,0.287229,ideal,ideal
3,1.336751,1.750000,acceptable,unacceptable
4,-0.584689,0.359022,unacceptable,acceptable
5,0.098145,-0.021054,ideal,acceptable
6,-0.258972,0.066109,ideal,acceptable
7,0.803090,0.651708,acceptable,ideal
8,0.151093,0.004787,ideal,acceptable
9,0.333958,0.113097,ideal,acceptable
